In [1]:
# import sys
# !{sys.executable} -m pip install <library_name> --user

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

C:\Users\felip\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\felip\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\felip\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [32]:
df = pd.read_csv("trainingset_v1d1_metadata.csv")
df = df[(df["sample_type"] == 'train') | (df["sample_type"] == 'validation')]
df["label"].value_counts()
df["snr"].describe()
df["event_time"].describe()
print(df["label"].value_counts())

Blip                   1549
Koi_Fish                604
Low_Frequency_Burst     531
Light_Modulation        434
Extremely_Loud          383
Low_Frequency_Lines     381
Power_Line              381
Scattered_Light         376
Violin_Mode             348
Scratchy                287
1080Lines               278
Whistle                 254
Helix                   237
Repeating_Blips         226
No_Glitch               129
Tomte                    90
1400Ripples              72
None_of_the_Above        70
Chirp                    50
Air_Compressor           49
Wandering_Line           35
Paired_Doves             23
Name: label, dtype: int64


In [33]:
from sklearn.ensemble import ExtraTreesClassifier

features_list = ['duration', 'peak_frequency', 'central_freq', 'bandwidth', 'amplitude', 'snr']

features = df[features_list]
labels = np.array([l for l in df["label"]]) # deep copy
labels[(labels == 'No_Glitch') | (labels == 'None_of_the_Above')] = '0'
labels[labels != '0'] = '1'
labels = labels.astype('int32')

num_runs = 5
avg_importance_scores_df = pd.DataFrame(index=features_list)

for i in range(num_runs):
    importance_model = ExtraTreesClassifier()
    importance_model.fit(features, labels)
    feat_importances = pd.DataFrame(importance_model.feature_importances_, index=features_list, columns=["Scores"])
    avg_importance_scores_df["Run " + str(i)] = feat_importances

avg_importance_scores_df = np.round(np.mean(avg_importance_scores_df, axis=1), 3)
avg_importance_scores_df = avg_importance_scores_df.sort_values(ascending=True)

fig = px.bar(feat_importances, x=avg_importance_scores_df, y=avg_importance_scores_df.index, orientation='h', range_x=[0, 0.3])
fig.update_layout(title="Normalized Feature Importance to Flag a Signal as a Glitch",
                  xaxis_title="",
                  yaxis_title="Importance Score")
fig.show()

In [52]:
glitch_means = list()
df_mean = pd.DataFrame(columns=["glitch"] + features_list)

for g in df["label"].unique():
    df_g = df[df["label"] == g]
    df_g = df_g[features_list]
    df_g = np.mean(df_g)
    df_g["glitch"] = g
    df_mean.loc[len(df_mean)] = df_g
    
df_mean

,glitch,duration,peak_frequency,central_freq,bandwidth,amplitude,snr
0,Power_Line,0.747151,61.992825,750.986601,1403.813546,1.134146e-21,11.328077
1,Scratchy,1.495841,152.672804,1251.172702,2327.443468,8.766585e-23,8.666357
2,1080Lines,0.828513,1104.075073,2993.923637,4800.451073,2.446187e-22,10.138611
3,Helix,0.084486,134.375244,252.218258,304.572613,3.544025e-22,8.852982
4,Violin_Mode,0.300070,1663.953540,1731.257748,2599.740972,2.627500e-20,13.465824
5,Air_Compressor,0.410196,48.203198,342.185632,611.896954,2.490510e-22,8.652775
6,No_Glitch,2.006683,187.208393,1643.855612,2988.191382,3.470196e-20,9.306449
7,Whistle,0.591901,1102.633167,2662.928971,4745.760010,1.436699e-21,9.424398
8,Low_Frequency_Lines,4.024080,11.955709,2628.428438,5240.822203,6.266124e-19,23.451611
9,Blip,0.278751,199.182432,832.732766,1582.720197,2.863826e-21,22.780742


In [54]:
glitches = ['Blip', 'Koi_Fish']
feature_to_plot = 'amplitude'
x = 0.9 # snr cutoff

range_x = 5e-19
fig = go.Figure()
for g in glitches:
    df_g = df[df['label'] == g]
    snr_list = np.sort(df_g["snr"])
    snr_cutoff = (snr_list[int(x*len(snr_list)):])[0] # find 'x'th percentile cutoff point
    hist = go.Histogram(x=df_g[(df_g["snr"] >= snr_cutoff) & (df_g[feature_to_plot] <= range_x)][feature_to_plot], nbinsx=20, name=str(g))
    fig.add_trace(hist)
fig.update_layout(title=f"{glitches} {feature_to_plot}, range: [0, {range_x}], {100-(100*x)}% Loudest SNRs only")
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

range_x = 5e-21
fig = go.Figure()
for g in glitches:
    df_g = df[df['label'] == g]
    snr_list = np.sort(df_g["snr"])
    snr_cutoff = (snr_list[int(x*len(snr_list)):])[0] # find 'x'th percentile cutoff point
    hist = go.Histogram(x=df_g[(df_g["snr"] >= snr_cutoff) & (df_g[feature_to_plot] <= range_x)][feature_to_plot], nbinsx=50, name=str(g))
    fig.add_trace(hist)
fig.update_layout(title=f"{glitches} {feature_to_plot}, range: [0, {range_x}], {100-(100*x)}% Loudest SNRs only")
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

In [61]:
glitches = ['Blip', 'Koi_Fish']
feature_to_plot = 'peak_frequency'
x = 0.9 # snr cutoff

range_x = 600
fig = go.Figure()
for g in glitches:
    df_g = df[df['label'] == g]
    snr_list = np.sort(df_g["snr"])
    snr_cutoff = (snr_list[int(x*len(snr_list)):])[0] # find 'x'th percentile cutoff point
    hist = go.Histogram(x=df_g[(df_g["snr"] >= snr_cutoff) & (df_g[feature_to_plot] <= range_x)][feature_to_plot], nbinsx=20, name=str(g))
    fig.add_trace(hist)
fig.update_layout(title=f"{glitches} {feature_to_plot}, range: [0, {range_x}], {100-(100*x)}% Loudest SNRs only")
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

range_x = 350
fig = go.Figure()
for g in glitches:
    df_g = df[df['label'] == g]
    snr_list = np.sort(df_g["snr"])
    snr_cutoff = (snr_list[int(x*len(snr_list)):])[0] # find 'x'th percentile cutoff point
    hist = go.Histogram(x=df_g[(df_g["snr"] >= snr_cutoff) & (df_g[feature_to_plot] <= range_x)][feature_to_plot], nbinsx=20, name=str(g))
    fig.add_trace(hist)
fig.update_layout(title=f"{glitches} {feature_to_plot}, range: [0, {range_x}], {100-(100*x)}% Loudest SNRs only")
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()